# Introduction

In this notebook we use deep learning techniques on the dataset we earlier created to create full end-to-end data science pipeline.

# Libraries

## Installation

In [ ]:
# Theano
# Keras

# Image Dataset

In [ ]:
### Load the dataset
### Import image

## Canonicalizing Images

All the images are of size 640x640. Although the images are all of same resolution, we further slice the image into 9 or 16 equal portions.

- 2x2 split = 320x320 size
- 3x3 split = 213.33x213.33 size
- 4x4 split = 160x160 size

# Train / Validate / Test splits for large datasets

- Get the Target Label (Bins of median income for classification)
- Generate the image data matrix that is to be input into VGG

# VGG

In [ ]:
# Use a pretrained VGG-16 network to create useful features


# Feature Generation